In [90]:
import os
import pandas as pd
import sklearn
from sklearn.feature_extraction.text import CountVectorizer  
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

In [100]:
merged = pd.read_csv("merged_with_gender.tsv", delimiter="\t")
# filter out examples where we failed to infer gender
genders = ['male', 'female']
df_filtered = merged[merged['gender'].isin(genders)]
df_filtered.shape

(53401, 11)

Stolen from stanford people: https://raw.githubusercontent.com/stanfordnlp/CoreNLP/master/data/edu/stanford/nlp/patterns/surface/stopwords.txt

In [101]:
stop_words = [w.strip() for w in open("stopwords.txt").readlines()]
stop_words[:10]

['!!', '?!', '??', '!?', '`', '``', "''", '-lrb-', '-rrb-', '-lsb-']

Let's remove gender pronouns since these aren't terribly interesting as predictors

In [118]:
gender_pronouns = ["she", "her", "hers", "his", "he", "guy", "she's", "he's", "commentshe", "commentsshe", "man", "woman"]
stop_words.extend(gender_pronouns)

Merge reviews for individual doctors (we have multiple reviews for each!)

In [105]:
unique_drs = df_filtered["dr-id"].unique()
unique_drs.shape
grouped = df_filtered.groupby('dr-id')
texts, sexes = [], []
for i, dr in grouped:
    cur_text = " ".join(dr['review-text'].values)
    texts.append(cur_text)
    cur_sex = dr["gender"].values[0]
    sexes.append(cur_sex)

In [110]:
len(texts)

16488

How many females v males?

In [109]:
sexes.count("male")/float(len(sexes))

0.7172489082969432

So about 72% male

In [121]:
vectorizer = CountVectorizer(max_features=20000, min_df=5, ngram_range=(1,1), binary=True, stop_words=stop_words)
X = vectorizer.fit_transform(texts)
y = sexes

Note; should probably pull out and pool reviews for physicians -- here we treat individual reviews for the same physician as multiple / independent instances

In [138]:
param_grid = {"C":[.01, .1, 1]}
clf = GridSearchCV(LogisticRegression(), param_grid=param_grid, scoring="f1_macro")

In [139]:
clf.fit(X,y)

GridSearchCV(cv=None, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1, param_grid={'C': [0.01, 0.1, 1]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='f1_macro', verbose=0)

In [140]:
clf.best_score_

0.54585116023791591

In [141]:
def show_most_informative_features(vectorizer, clf, n=50):
    feature_names = vectorizer.get_feature_names()
    coefs_with_fns = sorted(zip(clf.coef_[0], feature_names))
    top = zip(coefs_with_fns[:n], coefs_with_fns[:-(n + 1):-1])
    for (coef_1, fn_1), (coef_2, fn_2) in top:
        print ("\t%.4f\t%-15s\t\t%.4f\t%-15s" % (coef_1, fn_1, coef_2, fn_2))

In [142]:
show_most_informative_features(vectorizer, clf.best_estimator_, n=100)

	-1.9652	murray         		1.7321	vague          
	-1.8244	daiza          		1.6401	jerk           
	-1.7288	mary           		1.6075	haber          
	-1.6664	newman         		1.4948	cocky          
	-1.6497	erland         		1.4261	prostate       
	-1.6487	inpatient      		1.4230	adjusting      
	-1.6454	fahmi          		1.4014	decker         
	-1.6308	lanna          		1.3903	john           
	-1.6146	eclampsia      		1.3657	charles        
	-1.6016	rita           		1.3581	forseter       
	-1.5974	barber         		1.3566	introduce      
	-1.5827	fauzia         		1.3365	zager          
	-1.5634	gallo          		1.3307	endoscopy      
	-1.5625	nahra          		1.3165	abruptly       
	-1.5536	nakra          		1.3041	covering       
	-1.5406	experince      		1.3017	pursue         
	-1.5388	jahan          		1.2946	valley         
	-1.5385	uninformed     		1.2935	playing        
	-1.5242	incorrect      		1.2840	2x             
	-1.5107	manors         		1.2706	agitated       
	-1.4792	peggy      